# Miner's Arena

Welcome to the Miner's Arena where the blood runs thick and warriors' lives are cheap. Enter the pit for eternal glory or for eternal sleep.

## Rules of the Arena

The rules are simple. All warriors start the tournament with equal strength of 100 points. At each step, two warriors are randomly chosen and they perform a round of combat. Each warrior decides to perform one of the following actions:

- **FIGHT**: the warrior attacks the opponent
- **DEFEND**: the warrior actively defends herself/himself
- **FLEE**: the warrior runs away from the opponent

Depending on the combination of selected actions, the warriors sustain the following wounds (rows and first values represent the decision of the first warrior, respectively):

|            | **FIGHT** | **DEFEND** | **FLEE** |
|------------|:---------:|:----------:|:--------:|
| **FIGHT**  |  -15, -15 |   -2, -5   |  0, -10  |
| **DEFEND** |   -5, 2   |   -5, -5   |   0, -1  |
| **FLEE**   |   -10, 0  |    -1, 0   |  -1, -1  |

For instance, if the first warrior decides to **FIGHT** and the second warrior decides to **FLEE**, the first warrior does not sustain any wounds while the fleeing opponent loses 10 pts of strength.

When asked to make the decision, the warrior receives the full history of her/his opponent decisions. The history consists of tuples, where each tuple contains the strenght of the fighter the opponent was facing and the decision made by the opponent. For instance, the history `[(50, FIGHT), (98, DEFEND), (75, DEFEND)]` means that the opponent has participated in three rounds of combat, during the first round has decided to **FIGHT** (and the other fighter was down to the half of strength), and during the next two rounds has decided to **DEFEND** (in both cases other fighters were relatively strong). *Remember that during the first round of combat of a warrior, her/his history will be an empty list!*

Besides the history of opponent's fighting, the fighter receives also her/his own strength and the current strength of the opponent.

Your task is to prepare a strategy of combat. You don't know who will you face or how many times you will be selected to fight. Your goal is to survive until the end of the tournament. Remember: there can be only one! To make sure that the results are not due to chance, the tournament will be run 1000 times and the final ranking will be made based on the total number of wins.

## Example

Below you will see the example of a warrior and the way the tournament will be played. Read through the code, execute it, read the comments and make sure you understand it.

We begin by some obvious imports and the definition of constants.

In [ ]:
import numpy as np
from typing import List, Tuple, Callable

# actions
FIGHT = 1
DEFEND = 0
FLEE = -1

# typing hints
Strength = int
Decision = int
History = List[Tuple[Strength, Decision]]
Strategy = Callable[[Strength, Strength, History], Decision]

# combat outcomes
OUTCOMES = {
    (FIGHT, FIGHT): (-15, -15),
    (FIGHT, DEFEND): (-2, -5),
    (DEFEND, FIGHT): (-5, -2),
    (FIGHT, FLEE): (0, -10),
    (FLEE, FIGHT): (-10, 0),
    (DEFEND, DEFEND): (-5, -5),
    (DEFEND, FLEE): (0, -1),
    (FLEE, DEFEND): (-3, 0),
    (FLEE, FLEE): (-1, -1),
}

Next is the definition of the `Warrior` class

In [ ]:
class Warrior:
    def __init__(self, name: str, strategy: Strategy):
        self.name = name
        self.strategy = strategy
        self.history: History = []
        self.strength = 100

    def action(self, opponent_strength: Strength, opponent_history: History):
        return self.strategy(self.strength, opponent_strength, list(opponent_history))

    def engage(self, opponent):
        my_action = self.action(opponent.strength, opponent.history)
        opponent_action = opponent.action(self.strength, self.history)

        # update combat history by recording actions taken by the warriors during this turn
        self.history.append((opponent.strength, my_action))
        opponent.history.append((self.strength, opponent_action))

        # compute the wounds sustained by the warriors
        my_wound, opponent_wound = OUTCOMES[my_action, opponent_action]

        # update the strength of each warrior
        self.strength = max(0, self.strength + my_wound)
        opponent.strength = max(0, opponent.strength + opponent_wound)

    @property
    def is_alive(self):
        return self.strength > 0

Let's start with an example of a very simple strategy, where the warrior always flees from the combat. 

In [ ]:
def coward(own_strength, opponent_strength, history):
    """Always flees from the battlefield.
    
        >>> coward(100, 10, [])
        -1
        >>> coward(100, 10, [(100, 1)])
        -1
        >>> coward(100, 10, [(100, 1), (99, 1), (98, 1)])
        -1
    """
    return FLEE

You will implement a very similar function. It may be much more complex, it may involve machine learning, it may implement a simple heuristic, whatever works. Make sure to write at least these three [docstring tests](https://docs.python.org/3/library/doctest.html) to verify if your warrior can properly handle each of the three scenarios:
- the opponent hasn't fought yet, her/his history is empty
- the opponent has fought a single combat
- the opponent has the history of combat consisting of several rounds

To run docstring tests execute the following cell

In [ ]:
import doctest

doctest.testmod()

Here are three more examples of fighting strategies:
- *berserker*: always attacks
- *random warrior*: selects the action completely randomly
- *mirror warrior*: mirrors the most frequent action of the opponent

In [ ]:
def berserker(own_strength, opponent_strength, history):
    """Always attack, no matter the cost.
    
        >>> berserker(100, 10, [])
        1
        >>> berserker(100, 10, [(100, 1)])
        1
        >>> berserker(100, 10, [(100, 1), (99, 1), (98, 1)])
        1
    """
    return FIGHT

In [ ]:
def random_warrior(own_strength, opponent_strength, history):
    """Choose a random action.
    
        >>> random_warrior(100, 10, []) in [FLEE, DEFEND, FIGHT]
        True
        >>> random_warrior(100, 10, [(100, 1)]) in [FLEE, DEFEND, FIGHT]
        True
        >>> random_warrior(100, 10, [(100, 1), (99, 1), (98, 1)]) in [FLEE, DEFEND, FIGHT]
        True
    """
    return np.random.choice([FIGHT, DEFEND, FLEE])

In [ ]:
from statistics import mode

def mirror_warrior(own_strength, opponent_strength, history):
    """Choose the most frequent action of the opponent. If no history is available, choose a random action.
       Make sure to check what happens if two or more actions appear in the history with the same frequency!
    
        >>> mirror_warrior(100, 10, []) in [FLEE, DEFEND, FIGHT]
        True
        >>> mirror_warrior(100, 10, [(100, 1)])
        1
        >>> mirror_warrior(100, 10, [(100, 0), (99, 1), (98, 0), (98, 1)])
        0
    """
    if history:
        _, actions = zip(*history)
        return mode(actions)
    else:
        return np.random.choice([FIGHT, DEFEND, FLEE])

Finally, let's take a look at the Arena and at the mechanics of combat. In this example I am creating one warrior for each of the above strategies. In reality, the Arena will be populated by the warriors you implement. But I might add a few surprise warriors myself...

In [ ]:
class Arena:
    def __init__(self):
        self.warriors = []
        
    def add(self, warrior: Warrior):
        self.warriors.append(warrior)

    def turn(self):
        # randomly pick two warriors
        first_warrior, second_warrior = np.random.choice(self.warriors, size=2, replace=False)

        first_warrior.engage(second_warrior)

        # remove the warrior from the Arena if s/he was killed during this turn
        if not first_warrior.is_alive:
            print(f"{second_warrior.name} kills {first_warrior.name}")
            self.warriors.remove(first_warrior)
        if not second_warrior.is_alive:
            print(f"{first_warrior.name} kills {second_warrior.name}")
            self.warriors.remove(second_warrior)

    def tournament(self, max_number_of_turns: int):
        turn = 1

        while len(self.warriors) > 1 and turn <= max_number_of_turns:
            turn += 1
            self.turn()

        if len(self.warriors) == 1:
            print(f"There can be only one! {self.warriors[0].name}")
        elif len(self.warriors) == 0:
            print(f"All lying dead on the arena")
        else:
            print(f"Just cowards running around...")

The last step is to run the combat on the Arena. We limit the number of turns, because one possibility is that only cowards remain and they would never engage in combat. If the tournament ends with more than one warrior on the Arena (i.e. two or more cowards), or if all are lying dead (the last two warriors killed each other during the last turn), no one is declared the winner.

In [ ]:
arena = Arena()

arena.add(Warrior("Erik the Mouseheart", coward))
arena.add(Warrior("Harald the Thickhead", berserker))
arena.add(Warrior("Olaf the Clueless", random_warrior))
arena.add(Warrior("Rolf the Mimic", mirror_warrior))

arena.tournament(max_number_of_turns=1000)

## Assignment

Prepare your own function which will be used as your fighting strategy. The name of the function does not matter. You can use all standard libraries available through `pip`. Please abide by the following rules:

- your function cannot use self-inspection to try to manipulate your own strength (such attempts disqualify the warrior)
- your function cannot assume any particular name of the opponent (i.e. you cannot form coalitions, everyone fends for themselves)
- please make sure that your function works correctly via docstring tests

Send your function as a text attachment to Mikolaj.Morzy@put.poznan.pl. Put [arena] in the subject line of your message. As a bonus, you may ask for a particular name of your fighter. The deadline for submissions is **Sunday, June 11, 9pm**.